In [3]:
import pandas as pd
import geopandas as gpd
import os


In [4]:
folder = r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data"
file_name = r"rentals.csv"
geocoded_file_name = f"{file_name}"

dots_source = pd.read_csv(f"{folder}\{file_name}")

gdf = gpd.GeoDataFrame(
    dots_source, geometry=gpd.points_from_xy(dots_source.Longitude, dots_source.Latitude), crs="EPSG:4326"
)

In [1]:
def format_tractce(tract):
    tract_str = str(tract)
    if len(tract_str) == 3:
        return '000' #+ tract_str + '00'
    elif len(tract_str) == 4:
        return '00' + tract_str
    elif len(tract_str) == 5:
        return '0' + tract_str
    elif len(tract_str) == 6:
        return tract_str
    else:
        return tract_str  # In case of unexpected length


In [5]:
# demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source\tl_2023_53_bg.shp")

demographic_areas = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb")

demographic_areas["TRACT_FIPS"] = demographic_areas['TRACT_FIPS'].apply(format_tractce)

demographic_areas.to_crs("EPSG:4326", inplace=True)
demographic_areas["cbg_geoid"] = \
    demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)  \
    + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3) \
    + demographic_areas["TRACT_FIPS"].astype(str).str.zfill(6) \
    + demographic_areas["BLOCKGROUP_FIPS"].astype(str)

demographic_areas["TRACT_FIPS_SIMPLE"] = demographic_areas['TRACT_FIPS'].apply(format_tractce).str[:-2] + '00'

demographic_areas["cbg_geoid_simple_tractce"] = \
    demographic_areas["STATE_FIPS"].astype(str).str.zfill(2)  \
    + demographic_areas["COUNTY_FIPS"].astype(str).str.zfill(3) \
    + demographic_areas["TRACT_FIPS_SIMPLE"].astype(str) \
    + demographic_areas["BLOCKGROUP_FIPS"].astype(str)

demographic_areas = demographic_areas[["cbg_geoid","cbg_geoid_simple_tractce", "geometry"]]

cbsa_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp")
cbsa_source.to_crs("EPSG:4326", inplace=True)
cbsa_source = cbsa_source[["GEOID", "NAME", "geometry"]].rename(columns={"GEOID":"cbsa_geoid", "NAME": "cbsa_name"})

state_source = gpd.read_file(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp")
state_source.to_crs("EPSG:4326", inplace=True)
state_source = state_source[["FID", "State_Code", "geometry"]].rename(columns={"FID":"state_id", "State_Name": "state_name"})


KeyboardInterrupt: 

In [33]:
test = demographic_areas[["STATE_FIPS", "COUNTY_FIPS", "TRACT_FIPS", "TRACT_FIPS_SIMPLE", "BLOCKGROUP_FIPS" ]]

test[test["TRACT_FIPS"] != test["TRACT_FIPS_SIMPLE"]]

,STATE_FIPS,COUNTY_FIPS,TRACT_FIPS,TRACT_FIPS_SIMPLE,BLOCKGROUP_FIPS
10,01,001,020501,020500,1
11,01,001,020501,020500,2
12,01,001,020502,020500,1
13,01,001,020502,020500,2
14,01,001,020503,020500,1
...,...,...,...,...,...
239193,56,043,000301,000300,1
239194,56,043,000301,000300,2
239195,56,043,000301,000300,3
239196,56,043,000302,000300,1


In [25]:
gdf = gdf.drop(['cbg_geoid', 'cbsa_geoid', 'cbsa_name', 'state_id','State_Code'], axis =1 )

geocoded_dots = gdf.sjoin(demographic_areas, how = "left").drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(cbsa_source, how = 'left').drop(["index_right"], axis=1)
geocoded_dots = geocoded_dots.sjoin(state_source, how = 'left').drop(["index_right"], axis=1)

geocoded_dots = geocoded_dots.drop(['geometry'],axis=1)


# geocoded_dots.to_csv(f"{folder}/{geocoded_file_name}", index= False) 

Index(['Property ID', 'URL', 'Property Type', 'Address', 'City', 'State',
       'ZIP Code', 'Country Code', 'Latitude', 'Longitude', 'Rental ID',
       'Max Beds', 'Max Baths', 'Max Square Feet', 'Max Rent Price',
       'Description', 'updated_date', 'cbg_geoid', 'cbg_geoid_simple_tractce',
       'cbsa_geoid', 'cbsa_name', 'state_id', 'State_Code'],
      dtype='object')

,Property ID,URL,Property Type,Address,City,State,ZIP Code,Country Code,Latitude,Longitude,...,Max Square Feet,Max Rent Price,Description,updated_date,cbg_geoid,cbg_geoid_simple_tractce,cbsa_geoid,cbsa_name,state_id,State_Code
0,191142853,/ID/Pocatello/1833-W-Quinn-Rd-83202/unit-B/hom...,6,1833 W Quinn Rd Unit B,Pocatello,ID,83202.0,1,42.904825,-112.491625,...,1080.0,1000.0,NaN,NaN,160050015003,160050015003,38540,"Pocatello, ID",13.0,ID
1,191136919,/ID/Pocatello/1821-W-Quinn-Rd-83202/unit-A/hom...,6,1821 W Quinn Rd Unit A,Pocatello,ID,83202.0,1,42.904927,-112.490953,...,1080.0,1050.0,NaN,NaN,160050015003,160050015003,38540,"Pocatello, ID",13.0,ID
2,191053794,/ID/Pocatello/200-S-Main-St-83204/unit-I4/home...,6,200 S Main St Unit I4,Pocatello,ID,83204.0,1,42.861226,-112.449258,...,NaN,400.0,The Pocatello Coworking Space has many options...,NaN,160050016031,160050016001,38540,"Pocatello, ID",13.0,ID
3,179088685,/ID/Blackfoot/1020-S-Broadway-St-83221/unit-7/...,5,1020 S Broadway St Unit 7,Blackfoot,ID,83221.0,1,43.181143,-112.354596,...,NaN,500.0,Pet friendly studio apartment. Nice community ...,NaN,160119505023,160119505003,13940,"Blackfoot, ID",13.0,ID
4,190751609,/ID/Shoshone/421-Idaho-75-83352/home/190751609,6,421 Idaho 75,Shoshone,ID,83352.0,1,42.996801,-114.395794,...,NaN,1165.0,** Smoking/Vaping is not permitted. Thank you!...,NaN,160639501002,160639501002,NaN,NaN,13.0,ID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55387,109658664,/TX/Boerne/128-Destiny-78006/home/109658664,6,128 Destiny,Boerne,TX,78006.0,1,29.796085,-98.703871,...,1600.0,2550.0,Gorgeous single one-story home with 4 bedrooms...,2024-07-04,482599704042,482599704002,41700,"San Antonio-New Braunfels, TX",44.0,TX
55388,48788695,/TX/Boerne/10046-Saxet-Dr-78006/home/48788695,6,10046 Saxet Dr,Boerne,TX,78006.0,1,29.716440,-98.689695,...,2771.0,3950.0,Make yourself right at home with this fully fu...,2024-07-04,480291821022,480291821002,41700,"San Antonio-New Braunfels, TX",44.0,TX
55389,48507033,/TX/Boerne/140-Dusty-Corral-78006/home/48507033,6,140 Dusty Corral,Boerne,TX,78006.0,1,29.783848,-98.693886,...,2824.0,2500.0,Great home near schools and neighborhood ameni...,2024-07-04,482599704062,482599704002,41700,"San Antonio-New Braunfels, TX",44.0,TX
55390,188720704,/TX/Boerne/10616-Yellowtail-Blvd-78006/home/18...,6,10616 Yellowtail Blvd,Boerne,TX,78006.0,1,29.731282,-98.698187,...,2100.0,3100.0,"Never lived in, Beautiful, Brand new Meritage ...",2024-07-04,480291821022,480291821002,41700,"San Antonio-New Braunfels, TX",44.0,TX


In [30]:
epa_cbg_demo['TRACTCE'].str.len().value_counts()


,STATE_ABBR,STATE_FIPS,COUNTY_FIPS,STCOFIPS,TRACT_FIPS,BLOCKGROUP_FIPS,FIPS,POPULATION,POP_SQMI,SQMI,POPULATION_2020,POP20_SQMI,Shape_Length,Shape_Area,geometry,cbg_geoid,TRACT_FIPS_SIMPLE,cbg_geoid_simple,cbg_geoid_simple_tractce
0,AL,01,001,01001,020100,1,010010201001,572,344.6,1.66,575,346.4,0.110790,0.000412,"MULTIPOLYGON (((-86.49965 32.47549, -86.49478 ...",010010201001,020100,010010201001,010010201001
1,AL,01,001,01001,020100,2,010010201002,1186,551.6,2.15,1200,558.1,0.096859,0.000534,"MULTIPOLYGON (((-86.48128 32.47745, -86.48127 ...",010010201002,020100,010010201002,010010201002
2,AL,01,001,01001,020200,1,010010202001,960,1215.2,0.79,974,1232.9,0.062128,0.000197,"MULTIPOLYGON (((-86.47672 32.48917, -86.47616 ...",010010202001,020200,010010202001,010010202001
3,AL,01,001,01001,020200,2,010010202002,1068,2179.6,0.49,1081,2206.1,0.052095,0.000122,"MULTIPOLYGON (((-86.46538 32.47373, -86.46570 ...",010010202002,020200,010010202002,010010202002
4,AL,01,001,01001,020300,1,010010203001,2339,1559.3,1.50,2377,1584.7,0.089438,0.000372,"MULTIPOLYGON (((-86.45369 32.49191, -86.45363 ...",010010203001,020300,010010203001,010010203001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239198,WY,56,045,56045,951100,1,560459511001,1311,1.7,770.69,1346,1.7,2.500752,0.223951,"MULTIPOLYGON (((-104.46111 44.18001, -104.4611...",560459511001,951100,560459511001,560459511001
239199,WY,56,045,56045,951100,2,560459511002,1997,1.3,1586.48,1990,1.3,4.546745,0.459285,"MULTIPOLYGON (((-104.37549 44.18163, -104.3682...",560459511002,951100,560459511002,560459511002
239200,WY,56,045,56045,951300,1,560459513001,979,62.3,15.72,1005,63.9,0.315097,0.004558,"MULTIPOLYGON (((-104.21059 43.85134, -104.2105...",560459513001,951300,560459513001,560459513001
239201,WY,56,045,56045,951300,2,560459513002,1110,276.8,4.01,1122,279.8,0.178010,0.001162,"MULTIPOLYGON (((-104.15861 43.85253, -104.1509...",560459513002,951300,560459513002,560459513002
